In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import preprocessing

import joblib
import os

from datetime  import datetime

# Must check parameter Rigorously

In [37]:
algo='xgb'

strategyName='long'
predictionColName="Signal"

feat_path=r'D:\DataSets\FeatSelection\xgb_s50f15m_featSelect.xlsx'
folder_model=r'D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\final-model'
folder_result=r'D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\result-data'

col_preb_0='Prob_Sideway_0'
col_preb_1='Prob_Trend_1'

if strategyName=='long':   
        
     #3q20-1q21
     isFullFeatures=False
     test_path=r'D:\DataSets\UnSeenData\Long_TestUnSeen_0720_Now.csv'   
     model_file='long_S50M15_XGB-80x8x01_01-2007t12-2020_selectFeat_b111020_2226.pickle.date'   
    
    
     #Aprol21  
#      isFullFeatures=True
#      model_file='long_S50M15_XGB_01-2007T03-2021.pickle.dat'

elif strategyName=='short':
     test_path=r'D:\DataSets\UnSeenData\Short_TestUnSeen_0720_Now.csv'
     model_file='short_S50M15_XGB-160x8x005_01-2007t12-2020_selectFeat_b111020_2228.pickle.dat'
    
print(strategyName)
print(model_file)
print(test_path)


short
short_S50M15_XGB-160x8x005_01-2007t12-2020_allFeat_b111020_2228.pickle.dat
D:\DataSets\UnSeenData\Short_TestUnSeen_0720_Now.csv


In [38]:

model_postfix='.pickle.dat'

predicttime=datetime.now().strftime('-p%d%m%y_%H%M')


result_file=model_file.replace(model_postfix,f"{predicttime}.csv")

print('model folder: ',folder_model)
print('result predict folder: ',folder_result)

modelfile=f'{folder_model}\\{model_file}'
resultfile=f'{folder_result}\\{result_file}'

print('**************************************************************************************************************')
print('result file ==>',resultfile)
print('test file ==>',test_path)
print('model file ==>',modelfile)



model folder:  D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\final-model
result predict folder:  D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\result-data
**************************************************************************************************************
result file ==> D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\result-data\short_S50M15_XGB-160x8x005_01-2007t12-2020_allFeat_b111020_2228-p070521_1632.csv
test file ==> D:\DataSets\UnSeenData\Short_TestUnSeen_0720_Now.csv
model file ==> D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\final-model\short_S50M15_XGB-160x8x005_01-2007t12-2020_allFeat_b111020_2228.pickle.dat


In [39]:
datetime_col='Date/Time'
dataset =pd.read_csv(test_path,index_col=datetime_col, parse_dates=[datetime_col],dayfirst=True)

dataset.drop(columns= ['Ticker'],inplace=True)
dataset.index.rename('datetime', inplace=True)

dataset.tail(5)

,open,high,low,close,indy_ma-220,indy_ma-550,indy_hh-220,indy_ll-220,indy_mid-220,indy_hh2-550,...,indy_mid2-550,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-220_ma110,cate_2trend-550_ma110,cate_rannkHL550-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,,,,,
2021-05-06 11:15:00,934.5,936.8,934.2,936.6,1.01,1.02,1.03,0.98,0.0011,1.04,...,0.0011,-4.21,-7.14,2.92,29.92,1.01,2,0,2,2
2021-05-06 11:30:00,936.7,938.0,935.3,936.5,1.01,1.02,1.03,0.98,0.0011,1.04,...,0.0011,-4.19,-7.10,2.91,31.35,1.01,2,0,2,2
2021-05-06 11:45:00,936.6,937.5,935.5,935.6,1.01,1.02,1.03,0.98,0.0011,1.04,...,0.0011,-4.18,-7.07,2.89,32.83,1.02,2,0,2,2
2021-05-06 12:00:00,935.7,936.9,935.5,936.0,1.01,1.02,1.03,0.98,0.0011,1.04,...,0.0011,-4.16,-7.04,2.88,34.34,1.02,2,0,2,2
2021-05-06 12:15:00,935.9,936.5,934.4,936.4,1.01,1.02,1.03,0.98,0.0011,1.04,...,0.0011,-4.13,-7.01,2.87,35.47,1.01,2,0,2,2


In [40]:
select_feat=np.array(dataset.columns )

if  isFullFeatures==True:
    print("FullFeatures")
    select_feat=np.array(dataset.columns )
else:
    feat_path=os.path.abspath(feat_path)
    feat_df=pd.read_excel(feat_path,header=None,engine='openpyxl',sheet_name=sheet_name_feat)
    #print('feature selection : ',feat_df)
    print("Selective Features")
    select_feat=feat_df.loc[:,0].tolist()
    select_feat.extend(['open','high','low','close'])
    select_feat=np.array(select_feat )
    

print(type(select_feat))
print(select_feat)


FullFeatures
<class 'numpy.ndarray'>
['open' 'high' 'low' 'close' 'indy_ma-220' 'indy_ma-550' 'indy_hh-220'
 'indy_ll-220' 'indy_mid-220' 'indy_hh2-550' 'indy_ll2-550'
 'indy_mid2-550' 'indy_macd110-440' 'indy_signal110-440-110'
 'indy_hist_macd110-440' 'indy_rsi25-ma20' 'indy_6ATRTrail_DC-110'
 'cate_3trend-220_ma110' 'cate_2trend-550_ma110' 'cate_rannkHL550-ma66'
 'cate_CombineTrend']


# Filter train period

In [41]:
df_ohlc=dataset[['open','high','low','close']]
df_ohlc.info()
df_ohlc.tail()

df_test=dataset.drop(columns=['open','high','low','close'])
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4435 entries, 2020-07-01 09:45:00 to 2021-05-06 12:15:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4435 non-null   float64
 1   high    4435 non-null   float64
 2   low     4435 non-null   float64
 3   close   4435 non-null   float64
dtypes: float64(4)
memory usage: 173.2 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4435 entries, 2020-07-01 09:45:00 to 2021-05-06 12:15:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-220             4435 non-null   float64
 1   indy_ma-550             4435 non-null   float64
 2   indy_hh-220             4435 non-null   float64
 3   indy_ll-220             4435 non-null   float64
 4   indy_mid-220            4435 non-null   float64
 5   indy_hh2-550            4435 non-null   float64
 6   indy_ll2-550          

In [42]:
X_test=df_test.to_numpy()
print(len(X_test))
print(X_test[:5])


4435
[[ 1.010e+00  1.030e+00  1.040e+00  9.700e-01  1.100e-03  1.110e+00
   9.700e-01  1.200e-03 -3.386e+01 -2.394e+01 -9.920e+00  5.626e+01
   1.000e+00  1.000e+00  1.000e+00  3.000e+00  3.000e+00]
 [ 1.010e+00  1.030e+00  1.040e+00  9.700e-01  1.100e-03  1.110e+00
   9.700e-01  1.200e-03 -3.399e+01 -2.414e+01 -9.850e+00  5.589e+01
   1.000e+00  1.000e+00  1.000e+00  3.000e+00  3.000e+00]
 [ 1.010e+00  1.030e+00  1.040e+00  9.700e-01  1.100e-03  1.110e+00
   9.700e-01  1.200e-03 -3.411e+01 -2.434e+01 -9.760e+00  5.553e+01
   1.000e+00  1.000e+00  1.000e+00  3.000e+00  3.000e+00]
 [ 1.010e+00  1.030e+00  1.040e+00  9.700e-01  1.100e-03  1.110e+00
   9.700e-01  1.200e-03 -3.420e+01 -2.454e+01 -9.650e+00  5.522e+01
   1.000e+00  1.000e+00  1.000e+00  3.000e+00  3.000e+00]
 [ 1.010e+00  1.030e+00  1.040e+00  9.700e-01  1.100e-03  1.110e+00
   9.700e-01  1.200e-03 -3.429e+01 -2.475e+01 -9.540e+00  5.494e+01
   1.000e+00  1.000e+00  1.000e+00  3.000e+00  3.000e+00]]


# Load model from file to predict UnseenData or TestData



In [43]:

loaded_model = joblib.load(modelfile)
print(f"Loaded model from: {modelfile}")


Loaded model from: D:\JupyterCode\MyDev\AlgoTradingProject\ML-S50F\Lab-S50F_XGBoost\S50F-XGB_Model\final-model\short_S50M15_XGB-160x8x005_01-2007t12-2020_allFeat_b111020_2228.pickle.dat


# Test Data

In [44]:
yPredited = loaded_model.predict(X_test)
predictions = [round(value) for value in yPredited]


prediction_df=pd.DataFrame(data= {predictionColName: predictions },index=df_ohlc.index)

df_result=pd.concat([df_ohlc,prediction_df],axis=1)

print(df_result.info())
df_result.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4435 entries, 2020-07-01 09:45:00 to 2021-05-06 12:15:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4435 non-null   float64
 1   high    4435 non-null   float64
 2   low     4435 non-null   float64
 3   close   4435 non-null   float64
 4   Signal  4435 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 207.9 KB
None


,open,high,low,close,Signal
datetime,,,,,
2021-05-06 11:15:00,934.5,936.8,934.2,936.6,1
2021-05-06 11:30:00,936.7,938.0,935.3,936.5,1
2021-05-06 11:45:00,936.6,937.5,935.5,935.6,1
2021-05-06 12:00:00,935.7,936.9,935.5,936.0,1
2021-05-06 12:15:00,935.9,936.5,934.4,936.4,1


In [45]:
yPredited_prob = loaded_model.predict_proba(X_test)
yPredited_prob=yPredited_prob*100

df_predictProb=pd.DataFrame(data=yPredited_prob,columns=[col_preb_0,col_preb_1],index=df_ohlc.index)

df_predictProb.round(3).tail()


,Prob_Sideway_0,Prob_Trend_1
datetime,,
2021-05-06 11:15:00,33.116001,66.884003
2021-05-06 11:30:00,33.116001,66.884003
2021-05-06 11:45:00,36.418999,63.581001
2021-05-06 12:00:00,37.823002,62.176998
2021-05-06 12:15:00,39.821999,60.178001


In [46]:
if len(df_result)==len(df_predictProb):
 df=pd.concat([df_result,df_predictProb]  ,axis=1)
 df=df.drop(columns=[col_preb_0]) 
  
 print(df.tail(10))
else:
 print(Error)              
               

                      open   high    low  close  Signal  Prob_Trend_1
datetime                                                             
2021-05-06 10:00:00  925.2  927.9  924.0  926.4       1     99.358810
2021-05-06 10:15:00  926.5  930.3  925.9  928.0       1     99.370178
2021-05-06 10:30:00  928.1  930.5  927.7  930.3       1     99.423508
2021-05-06 10:45:00  930.2  932.3  929.2  930.3       1     99.423508
2021-05-06 11:00:00  930.4  934.7  930.3  934.4       1     66.883537
2021-05-06 11:15:00  934.5  936.8  934.2  936.6       1     66.883537
2021-05-06 11:30:00  936.7  938.0  935.3  936.5       1     66.883537
2021-05-06 11:45:00  936.6  937.5  935.5  935.6       1     63.580917
2021-05-06 12:00:00  935.7  936.9  935.5  936.0       1     62.177322
2021-05-06 12:15:00  935.9  936.5  934.4  936.4       1     60.177757


In [47]:
#-4= remove OHLC
if isFullFeatures==True:
 feat_title='full'
else:
 feat_title=f'feat{len(select_feat)-4}'

file_prediction=f'{algo}_{strategyName}_{feat_title}.csv'
df.to_csv(folder_result+"\\"+file_prediction)